In [19]:
from pathlib import Path
import geopandas as gpd
import rasterio
from rasterio.warp import transform, transform_geom
import shutil
from rasterio.mask import mask

layer_to_keep=(
    "0310-7670", "0310-7675",
    "0315-7665", "0315-7670", "0315-7675",
    "0320-7655", "0320-7660", "0320-7665",
    "0325-7650", "0335-7640", "0340-7640",
)

FOLDER_INPUT_2017=Path("data/ign/raw_data/ORTHOHR_1-0_RVB-0M20_JP2-E080_RGR92UTM40S_D974_2017-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2019-09-00386/OHR_RVB_0M20_JP2-E080_RGR92UTM40S_D974-2017")
FOLDER_OUTPUT_2017=Path("data/ign/IGN_2017_974")

if FOLDER_OUTPUT_2017.exists():
    shutil.rmtree(FOLDER_OUTPUT_2017)
FOLDER_OUTPUT_2017.mkdir(exist_ok=True, parents=True)


FOLDER_INPUT_2022=Path("data/ign/raw_data/BDORTHO_2-0_RVB-0M20_JP2-E080_RGR92UTM40S_D974_2022-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2024-03-00066/OHR_RVB_0M20_JP2-E080_RGR92UTM40S_D974-2022")
FOLDER_OUTPUT_2022=Path("data/ign/IGN_2022_974")

if FOLDER_OUTPUT_2022.exists():
    shutil.rmtree(FOLDER_OUTPUT_2022)
FOLDER_OUTPUT_2022.mkdir(exist_ok=True, parents=True)


emprise_lagoon_gpd = gpd.read_file("./data/emprise_lagoon.geojson")


In [20]:
# Function to clip raster to lagoon polygons
def clip_raster_to_polygons(input_raster: Path, output_raster: Path, polygons_gdf):
    if not input_raster.exists():
        print(f"File not found: {input_raster.name}")
        return

    with rasterio.open(input_raster) as src:
        raster_crs = src.crs
        print(f"Processing {input_raster.name} | CRS: {raster_crs}")

        # Reproject lagoon polygons to raster CRS
        polygons = [
            transform_geom(polygons_gdf.crs.to_string(), raster_crs.to_string(), geom)
            for geom in polygons_gdf.geometry
        ]

        # Clip raster
        out_image, out_transform = mask(src, polygons, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "compress": "DEFLATE",
            "tiled": True,
        })
        if out_meta.get("nodata") is None:
            out_meta["nodata"] = 0

    with rasterio.open(output_raster, "w", **out_meta) as dst:
        dst.write(out_image)

    print(f"Saved clipped raster: {output_raster}")


In [21]:
# 2017
for i, layer in enumerate(layer_to_keep):
    file_input=Path(f"{FOLDER_INPUT_2017}/974-2017-{layer}-U40S-0M20-E080.jp2")
    
    if not file_input.exists():
        print(f"File not found {file_input.name}")
        continue

    print(f"{i}/{len(layer_to_keep)}: {file_input.name}")
    
    file_output = Path(FOLDER_OUTPUT_2017, f"{file_input.stem}_clipped.tif")
    clip_raster_to_polygons(file_input, file_output, emprise_lagoon_gpd)


0/11: 974-2017-0310-7670-U40S-0M20-E080.jp2
Processing 974-2017-0310-7670-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2017_974/974-2017-0310-7670-U40S-0M20-E080_clipped.tif
1/11: 974-2017-0310-7675-U40S-0M20-E080.jp2
Processing 974-2017-0310-7675-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2017_974/974-2017-0310-7675-U40S-0M20-E080_clipped.tif
2/11: 974-2017-0315-7665-U40S-0M20-E080.jp2
Processing 974-2017-0315-7665-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2017_974/974-2017-0315-7665-U40S-0M20-E080_clipped.tif
3/11: 974-2017-0315-7670-U40S-0M20-E080.jp2
Processing 974-2017-0315-7670-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2017_974/974-2017-0315-7670-U40S-0M20-E080_clipped.tif
4/11: 974-2017-0315-7675-U40S-0M20-E080.jp2
Processing 974-2017-0315-7675-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2017_974/974-2017-0315-7675-U40S-0M20-E080_clipped.tif


In [22]:
# 2017
for i, layer in enumerate(layer_to_keep):
    file_input=Path(f"{FOLDER_INPUT_2022}/974-2022-{layer}-U40S-0M20-E080.jp2")
    
    if not file_input.exists():
        print(f"File not found {file_input.name}")
        continue
    
    print(f"{i}/{len(layer_to_keep)}: {file_input.name}")

    file_output = Path(FOLDER_OUTPUT_2022, f"{file_input.stem}_clipped.tif")
    clip_raster_to_polygons(file_input, file_output, emprise_lagoon_gpd)

0/11: 974-2022-0310-7670-U40S-0M20-E080.jp2
Processing 974-2022-0310-7670-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2022_974/974-2022-0310-7670-U40S-0M20-E080_clipped.tif
1/11: 974-2022-0310-7675-U40S-0M20-E080.jp2
Processing 974-2022-0310-7675-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2022_974/974-2022-0310-7675-U40S-0M20-E080_clipped.tif
2/11: 974-2022-0315-7665-U40S-0M20-E080.jp2
Processing 974-2022-0315-7665-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2022_974/974-2022-0315-7665-U40S-0M20-E080_clipped.tif
3/11: 974-2022-0315-7670-U40S-0M20-E080.jp2
Processing 974-2022-0315-7670-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2022_974/974-2022-0315-7670-U40S-0M20-E080_clipped.tif
4/11: 974-2022-0315-7675-U40S-0M20-E080.jp2
Processing 974-2022-0315-7675-U40S-0M20-E080.jp2 | CRS: EPSG:2975
Saved clipped raster: data/ign/IGN_2022_974/974-2022-0315-7675-U40S-0M20-E080_clipped.tif
